#  Tradução de Textos - Experimento
## Utilização do modelo [MarianMT](https://huggingface.co/transformers/model_doc/marian.html) para tradução. 


* Neste exempo a tradução é feito do inglês para o português, mas ela pode ser feita em qualquer uma das línguas suportadas pelo MarianMT. 
* Para adaptar para traduções em outras línguas é necessário verificacar se há o modelo pré treinado disponível no MarianMT e adaptar o truncamento de strings do [spacy](https://spacy.io/usage/models) para o idioma desejado
* A métrica computada é o [sacrebleu](https://https://github.com/mjpost/sacrebleu) 


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# dataset = "/tmp/data/paracrawl_en_pt_test.xlsx" #@param {type:"string"}
dataset = "translate.xlsx" 
text = "text" #@param {type:"string", label:"Atributo do texto", description:"Este atributo será traduzido e apresentado o resultado."}
target = "target" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será validado com os valores do alvo."}
input_language = "Inglês" #@param ["Alemão", "Catalão", "Espanhol", "Francês", "Inglês", "Italiano", "Latim", "Português", "Romeno"] {type:"string", label:"Idioma de entrada"}
output_language = "Português" #@param ["Alemão", "Catalão", "Espanhol", "Francês", "Inglês", "Italiano", "Latim", "Português", "Romeno"] {type:"string", label:"Idioma de saída"}

#Hyperparams 
seed = 42 #@param {type:"integer",label"Semente de aleatoriedade"}
input_max_length = 127 #@param {type:"integer",label"Tamanho máximo da sentença de entrada"}
output_max_length = 256 #@param {type:"integer",label"Tamanho máximo da sentença de saída"}
inference_batch_size = 2 #@param {type:"integer",label"Tamanho do Batch de inferência"}

In [ ]:
if input_language == output_language:
    raise Exception('Idioma de entrada e de saída não podem ser iguais')

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_excel(dataset)

## Formatar dados

Dados nulos serão removidos e serão separados em colunas de teste e validação.

In [ ]:
df = df.dropna()
X = df[text].to_numpy()

y = None
if target in df.columns:
    y = df[target].to_numpy()

## Verificando as configurações do MarianMT

- Verificando disponibilidade de GPU e status de hardware
- Instanciando modelo e tokenizador
- Opções de tradução de idiomas
- Modelos pré treinados disponíveis

In [ ]:
from multiprocessing import cpu_count
import torch

dev = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(dev)

if dev == "cpu":
    print(f"number of CPU cores: {cpu_count()}")
else:
    print(f"GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")

## Chamada da Classe MarianMT

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/main/tasks/nlp-marianmt-translator/marianmt_model.py

In [ ]:
from marianmt_model import MarianMTTranslator

marian_model = None
models = []

hyperparams = {'input_max_length': input_max_length, 
                'output_max_length': output_max_length,
                'inference_batch_size': inference_batch_size,
                'seed':seed}

prefixes = {"Alemão": ">>de<<",
            "Catalão": ">>ca<<",
            "Espanhol": ">>es<<",
            "Francês": ">>fr<<",
            "Inglês": ">>en<<",
            "Italiano": ">>it<<",
            "Latim": ">>la<<",
            "Português": ">>pt_br<<",
            "Romeno": ">>ro<<"}

prefix = prefixes[output_language]

In [ ]:
model_name_one, model_name_two = None, None

if input_language == "Inglês" or output_language == "Inglês":
    
    if input_language == "Alemão":
        model_name_one = "Helsinki-NLP/opus-mt-de-en"
    elif output_language == "Alemão":
        model_name_one = "Helsinki-NLP/opus-mt-en-de"
    else:
        model_name_one = "Helsinki-NLP/opus-mt-en-ROMANCE" if input_language == "Inglês" else "Helsinki-NLP/opus-mt-ROMANCE-en"

else:
    
    if input_language == "Alemão":
        model_name_one = "Helsinki-NLP/opus-mt-de-en"
    else:
        model_name_one = "Helsinki-NLP/opus-mt-ROMANCE-en"
    
    if output_language == "Alemão":
        model_name_two = "Helsinki-NLP/opus-mt-en-de"
    else:
        model_name_two = "Helsinki-NLP/opus-mt-en-ROMANCE"

In [ ]:
if input_language == "Inglês" or output_language == "Inglês":
    
    model_params = {'model_name': model_name_one,
                'prefix': prefix}
    
    marian_model = MarianMTTranslator(hyperparams, model_params)
    
    models.append(marian_model)

In [ ]:
if input_language != "Inglês" and output_language != "Inglês":
    
    model_params = {'model_name': model_name_one,
                'prefix': prefixes['Inglês']}
    
    marian_model = MarianMTTranslator(hyperparams, model_params)
    
    models.append(marian_model)
    
    X = marian_model.predict(X)
    
    model_params = {'model_name': model_name_two,
                'prefix': prefix}
    
    marian_model = MarianMTTranslator(hyperparams, model_params)
    
    models.append(marian_model)

In [ ]:
if y is not None:
    aux = marian_model.get_result_dataframe(X,y)
else:
    aux = marian_model.predict(X)

In [ ]:
aux

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_metrics

if y is not None:
    save_metrics(avg_bleu=marian_model.avg_bleu)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "models": models
}

dump(artifacts, "/tmp/data/translate.joblib")